#### import lib

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role
import tensorflow as tf
from sagemaker.tensorflow import TensorFlow

In [2]:
# test the version of tensorflow
print('Tensorflow ver: ' + tf.__version__)

Tensorflow ver: 2.2.2


In [3]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [4]:
PROCESS_DATA_PATH = 'Data/D1/processes'
EXPERIMENT_NO = 'Experiment/E1'

BASE_S3_URI="s3://ehr-model"
CLOUD_JOBNAME = f'model-training-cloud'
PROJECT_SRC_DIR = os.path.join(os.getcwd(), 'tensorflow_training')


In [5]:

S3_LOG_DIR = os.path.join(BASE_S3_URI, 'logs')
print(f'Log Directory >>> {S3_LOG_DIR}')

S3_DATA_DIR = os.path.join(BASE_S3_URI,PROCESS_DATA_PATH)
print(f'Raw Directory >>> {S3_DATA_DIR}')

# S3_RAW_JSON_DIR = os.path.join(BASE_S3_URI, 'data', RAW_DATA_NO, TYPE, POSE, JSON_DIR_NAME)
# print(f'Raw Directory >>> {S3_RAW_JSON_DIR}')

S3_MODEL_DIR = os.path.join(BASE_S3_URI, EXPERIMENT_NO)
print(f'Model Directory >>> {S3_MODEL_DIR}')


Log Directory >>> s3://ehr-model/logs
Raw Directory >>> s3://ehr-model/Data/D1/processes
Model Directory >>> s3://ehr-model/Experiment/E1


In [ ]:
## local execution
!python tensorflow_training/training.py --train "./" --model_output "model" --tf_logs_path "logs"

In [6]:
# Common parameters between Local / Cloud mode training
# path to the training script
training_script = 'training.py'
source_dir = 'tensorflow_training'
tf_framework_version = '2.2.0'
py_version = 'py37'
requirements_file = 'requirements.txt'

In [7]:
# Hyper parameters for training in local mode
hyperparameters = {
    'input_size': 3, # depend on number of feature
    'version': 2, #change it manually 
    'test_size': 0.15,
    'optimizer': 'adam',
    'epochs': 500, 
    'batch_size' : 18, 
    'learning_rate': 1e-3,
    'tf_logs_path': S3_LOG_DIR, 
}

# training paths for training in local mode.
inputs = {
    'train': S3_DATA_DIR,
}

# training_instance= ['ml.t2.medium']
training_instance= ['ml.m5.2xlarge','ml.c5.4xlarge','ml.p2.xlarge']# GPU ml.p2.xlarge
training_instance_count = 1

# Run the training in Cloud Mode.
estimator = TensorFlow(base_job_name=CLOUD_JOBNAME,
                       entry_point=training_script,
                       source_dir=source_dir,
                       role=role,
                       output_path=f'{S3_MODEL_DIR}/models/{hyperparameters["version"]}',
                       checkpoint_s3_uri=f'{S3_MODEL_DIR}/checkpoints/{hyperparameters["version"]}',
                       code_location=f'{S3_MODEL_DIR}/src',
                       framework_version=tf_framework_version,
                       py_version=py_version,
                       hyperparameters=hyperparameters,
                       train_instance_count=training_instance_count,
                       train_instance_type=training_instance[0], 
                       requirements_file=requirements_file)


estimator.fit(inputs)


Legacy mode is deprecated in versions 1.13 and higher. Using script mode instead. Legacy mode and its training parameters will be deprecated in SageMaker Python SDK v2. Please use TF 1.13 or higher and script mode.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2021-02-20 16:15:32 Starting - Starting the training job...
2021-02-20 16:15:34 Starting - Launching requested ML instances......
2021-02-20 16:16:43 Starting - Preparing the instances for training...
2021-02-20 16:17:23 Downloading - Downloading input data
2021-02-20 16:17:23 Training - Downloading the training image..2021-02-20 16:17:47.463547: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-02-20 16:17:47.466671: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-02-20 16:17:47.550197: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-02-20 16:17:50,387 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-02-20 16:17:50,394 sagemaker-training-toolkit INFO     No GPUs detected (normal if

#### Deployment for s3

In [36]:

predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')
print("EndPoint : "+ predictor.endpoint)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


---------------!EndPoint : model-training-cloud-2021-02-20-14-42-11-085


In [ ]:
### invoke endpoint 

In [19]:
import boto3
import json
runtime = boto3.Session().client(service_name='sagemaker-runtime', region_name='us-west-2')

gender= int(input("enter gender 1 - male 0 female"))
age= int(input("enter age "))
thalach= int(input("enter thalach \n"))
response = runtime.invoke_endpoint(
                    EndpointName="model-training-cloud-2021-02-20-14-42-11-085",
                    ContentType='application/json',
                    Body=json.dumps(([gender,age,thalach]))
                )
print("Prediction \n")
pred=json.loads(response['Body'].read().decode())
print(pred['predictions'][0][0])

enter gender 1 - male 0 female 1
enter age  23
enter thalach 125


Prediction
[93.0161057]


In [18]:
pred

'{\n    "predictions": [[96.5605164]\n    ]\n}'

In [ ]:
# training.py --batch_size 18 --epochs 500 --input_size 3 --learning_rate 0.001 --model_dir s3://ehr-model/Experiment/E1/models/1/model-training-cloud-2021-02-20-14-17-47-692/model --optimizer adam --test_size 0.15 --tf_logs_path s3://ehr-model/logs --version 1"